In [2]:
import requests
import json
import pandas as pd

In [ ]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',

}

url = 'https://eleapi.interno.gov.it/siel/PX/getentiR/DE/20241027/TE/07'

response = requests.get(url, headers=headers)
data = response.json()
data

{'int': {'file': 'GEOPOLITICA REGIONALI',
  'area': 'I',
  't_ele': 'Regionali Speciali',
  'dt_ele': 20241027000000},
 'enti': [{'cod': '070000000000',
   'desc': 'LIGURIA',
   'tipo': 'RE',
   'tipo_comune': None,
   'dt_agg': 20241029123920,
   'tipo_tras': None},
  {'cod': '070340000000',
   'desc': 'GENOVA',
   'tipo': 'CR',
   'tipo_comune': None,
   'dt_agg': 20241029123920,
   'tipo_tras': None},
  {'cod': '070340340000',
   'desc': 'GENOVA',
   'tipo': 'PR',
   'tipo_comune': None,
   'dt_agg': 20241029123920,
   'tipo_tras': None},
  {'cod': '070340340010',
   'desc': 'ARENZANO',
   'tipo': 'CM',
   'tipo_comune': 'N',
   'dt_agg': 20241028200750,
   'tipo_tras': 'SZ'},
  {'cod': '070340340020',
   'desc': 'AVEGNO',
   'tipo': 'CM',
   'tipo_comune': 'N',
   'dt_agg': 20241028194727,
   'tipo_tras': 'SZ'},
  {'cod': '070340340030',
   'desc': 'BARGAGLI',
   'tipo': 'CM',
   'tipo_comune': 'N',
   'dt_agg': 20241028220117,
   'tipo_tras': 'SZ'},
  {'cod': '070340340040',
   'd

In [6]:
enti_records = []

enti = data['enti']

for ente in enti:
    codice = ente.get('cod')
    descrizione = ente.get('desc')
    tipo = ente.get('tipo')
    tipo_comune = ente.get('tipo_comune')
    dt_agg = ente.get('dt_agg')
    tipo_tras = ente.get('tipo_tras')
    
    # Append each record as a dictionary
    enti_records.append({
        'codice': codice,
        'descrizione': descrizione,
        'tipo': tipo,
        'tipo_comune': tipo_comune,
        'dt_agg': dt_agg,
        'tipo_tras': tipo_tras
    })

# Convert the list of records into a DataFrame
df = pd.DataFrame(enti_records)
df.head(4)

,codice,descrizione,tipo,tipo_comune,dt_agg,tipo_tras
0,070000000000,LIGURIA,RE,None,20241029123920,None
1,070340000000,GENOVA,CR,None,20241029123920,None
2,070340340000,GENOVA,PR,None,20241029123920,None
3,070340340010,ARENZANO,CM,N,20241028200750,SZ


In [7]:
df['codice_regione'] = df['codice'].str[:2]
df['codice_circoscrizione'] = df['codice'].str[:2] #uguale a codice regione?
df['codice_provincia'] = df['codice'].str[5:8]
df['codice_comune'] = df['codice'].str[8:]

df['url'] = df.apply(
    lambda row: f'https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241027/TE/{row["codice_regione"]}/RE/{row["codice_circoscrizione"]}/PR/{row["codice_provincia"]}/CM/{row["codice_comune"]}',
    axis=1
)

df.sample(5)

,codice,descrizione,tipo,tipo_comune,dt_agg,tipo_tras,codice_regione,codice_circoscrizione,codice_provincia,codice_comune,url
102,070370370300,ISOLABONA,CM,N,20241028202054,SZ,07,07,037,0300,https://eleapi.interno.gov.it/siel/PX/getprefe...
228,070740740550,SASSELLO,CM,N,20241028201137,SZ,07,07,074,0550,https://eleapi.interno.gov.it/siel/PX/getprefe...
159,070390390200,LUNI,CM,N,20241028205953,SZ,07,07,039,0200,https://eleapi.interno.gov.it/siel/PX/getprefe...
140,070390390010,AMEGLIA,CM,N,20241028223329,SZ,07,07,039,0010,https://eleapi.interno.gov.it/siel/PX/getprefe...
130,070370370560,TERZORIO,CM,N,20241028170040,SZ,07,07,037,0560,https://eleapi.interno.gov.it/siel/PX/getprefe...


In [ ]:
df.to_csv('output/codici_liguria.csv', index=False, encoding='UTF-8-sig')